In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import nltk
import re
import string 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
import tensorflow as tf

In [2]:
train_data = pd.read_csv('train.tsv',sep="\t")
#val_data = pd.read_csv('valid.tsv',sep="\t")

In [3]:
train_data.columns=['id','label','statement','subjects','speaker','speaker_job_title','state_info','party_affiliation','count_1','count_2','count_3','count_4','count_5','context']

In [4]:
for i in range(0,len(train_data)):
  if((train_data['label'][i]=='true')):
    train_data['label'][i]=1
  else:
    train_data['label'][i]=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
#data = pd.concat([train_data, val_data], ignore_index=True, sort=False)
data=train_data
data.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [6]:
data['text']= data['statement'] + " " + data['subjects'] + " " + data['context']
del data['statement']
del data['id']
del data['subjects']
del data['context']
del data['speaker']
del data['speaker_job_title']
del data['state_info']
del data['party_affiliation']
del data['count_1']
del data['count_2']
del data['count_3']
del data['count_4']
del data['count_5']
data.head()

,label,text
0,0,When did the decline of coal start? It started...
1,0,"Hillary Clinton agrees with John McCain ""by vo..."
2,0,Health care reform legislation is likely to ma...
3,0,The economic turnaround started at the end of ...
4,1,The Chicago Bears have had more starting quart...


In [7]:
data.dropna()
data.fillna("", inplace=True)

In [8]:
pip install bs4

In [9]:
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
nltk.download("stopwords")   
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)

#Removal of stopwords 
def remove_stopwords_and_lemmatization(text):
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
            lemma = nltk.WordNetLemmatizer()
            word = lemma.lemmatize(word) 
            final_text.append(word)
    return " ".join(final_text)

#Total function
def cleaning(text):
    text = remove_characters(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

#Apply function on text column
data['text']=data['text'].apply(cleaning)

In [11]:
data=data.sample(frac=1)

#Training Model when data is 1% labeled

In [ ]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=42)

In [ ]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.99,random_state=0)

In [ ]:
max_features = 10000
maxlen = 200

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [ ]:
batch_size = 128

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units=160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units , return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate =.001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model=build_model()

In [ ]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
1/1 [==============================] - 12s 12s/step - loss: 0.6939 - accuracy: 0.3214 - val_loss: 0.6847 - val_accuracy: 0.8400
Epoch 2/300
1/1 [==============================] - 3s 3s/step - loss: 0.6827 - accuracy: 0.8750 - val_loss: 0.6697 - val_accuracy: 0.8400
Epoch 3/300
1/1 [==============================] - 3s 3s/step - loss: 0.6660 - accuracy: 0.8750 - val_loss: 0.6425 - val_accuracy: 0.8400
Epoch 4/300
1/1 [==============================] - 4s 4s/step - loss: 0.6351 - accuracy: 0.8750 - val_loss: 0.5898 - val_accuracy: 0.8400
Epoch 5/300
1/1 [==============================] - 6s 6s/step - loss: 0.5761 - accuracy: 0.8750 - val_loss: 0.4929 - val_accuracy: 0.8400
Epoch 6/300
1/1 [==============================] - 5s 5s/step - loss: 0.4628 - accuracy: 0.8750 - val_loss: 0.4610 - val_accuracy: 0.8400
Epoch 7/300
1/1 [==============================] - 5s 5s/step - loss: 0.3638 - accuracy: 0.8750 - val_loss: 0.6390 - val_accuracy: 0.8400
Epoch 8/300
1/1 [===============

In [ ]:
pred=(model.predict(X_unl) > 0.4).astype("int32")

In [ ]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [ ]:
model=build_model()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
45/45 [==============================] - 168s 4s/step - loss: 0.1387 - accuracy: 0.9766 - val_loss: 0.0902 - val_accuracy: 0.9821
Epoch 2/100
45/45 [==============================] - 160s 4s/step - loss: 0.0772 - accuracy: 0.9853 - val_loss: 0.1009 - val_accuracy: 0.9821
Epoch 3/100
45/45 [==============================] - 161s 4s/step - loss: 0.0803 - accuracy: 0.9853 - val_loss: 0.0914 - val_accuracy: 0.9821
Epoch 4/100
45/45 [==============================] - 160s 4s/step - loss: 0.0775 - accuracy: 0.9853 - val_loss: 0.0911 - val_accuracy: 0.9821
Epoch 5/100
45/45 [==============================] - 160s 4s/step - loss: 0.0771 - accuracy: 0.9853 - val_loss: 0.0910 - val_accuracy: 0.9821
Epoch 6/100
45/45 [==============================] - 160s 4s/step - loss: 0.0771 - accuracy: 0.9853 - val_loss: 0.0933 - val_accuracy: 0.9821
Epoch 7/100
45/45 [==============================] - 160s 4s/step - loss: 0.0767 - accuracy: 0.9853 - val_loss: 0.0875 - val_accuracy: 0.9821
Epoch 

In [ ]:
test_res=(model.predict(X_test) > 0.4).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [ ]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1695


In [ ]:
accuracy_1_labled=(count/len(test_res))*100

In [ ]:
print("Accuracy at 1% labled data",accuracy_1_labled)

Accuracy at 1% labled data 82.763671875


In [ ]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.84      0.99      0.91      1714
        Real       0.17      0.01      0.03       334

    accuracy                           0.83      2048
   macro avg       0.50      0.50      0.47      2048
weighted avg       0.73      0.83      0.76      2048



#Training Model at 1% labeled data and lr=.0001

In [ ]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=0)

In [ ]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.99,random_state=0)

In [ ]:
max_features = 10000
maxlen = 200

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [ ]:
batch_size = 128

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate =.0001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model=build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 200, 160)          167040    
                                                                 
 lstm_1 (LSTM)               (None, 160)               205440    
                                                                 
 dense (Dense)               (None, 160)               25760     
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 1,398,401
Trainable params: 398,401
Non-trainable params: 1,000,000
_________________________________________________________________


In [ ]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
1/1 [==============================] - 8s 8s/step - loss: 0.6942 - accuracy: 0.3571 - val_loss: 0.6926 - val_accuracy: 0.7200
Epoch 2/300
1/1 [==============================] - 2s 2s/step - loss: 0.6928 - accuracy: 0.6429 - val_loss: 0.6919 - val_accuracy: 0.6800
Epoch 3/300
1/1 [==============================] - 2s 2s/step - loss: 0.6916 - accuracy: 0.7857 - val_loss: 0.6913 - val_accuracy: 0.6800
Epoch 4/300
1/1 [==============================] - 2s 2s/step - loss: 0.6897 - accuracy: 0.8214 - val_loss: 0.6906 - val_accuracy: 0.6800
Epoch 5/300
1/1 [==============================] - 2s 2s/step - loss: 0.6882 - accuracy: 0.8929 - val_loss: 0.6898 - val_accuracy: 0.6800
Epoch 6/300
1/1 [==============================] - 2s 2s/step - loss: 0.6866 - accuracy: 0.8929 - val_loss: 0.6891 - val_accuracy: 0.6800
Epoch 7/300
1/1 [==============================] - 2s 2s/step - loss: 0.6854 - accuracy: 0.8929 - val_loss: 0.6883 - val_accuracy: 0.6800
Epoch 8/300
1/1 [=================

In [ ]:
pred=(model.predict(X_unl) > 0.5).astype("int32")

In [ ]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [ ]:
model=build_model()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
45/45 [==============================] - 192s 4s/step - loss: 0.4170 - accuracy: 0.9857 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 2/100
45/45 [==============================] - 179s 4s/step - loss: 0.0181 - accuracy: 0.9976 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/100
45/45 [==============================] - 180s 4s/step - loss: 0.0172 - accuracy: 0.9976 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 4/100
45/45 [==============================] - 179s 4s/step - loss: 0.0172 - accuracy: 0.9976 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 5/100
45/45 [==============================] - 178s 4s/step - loss: 0.0171 - accuracy: 0.9976 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 6/100
45/45 [==============================] - 178s 4s/step - loss: 0.0172 - accuracy: 0.9976 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 7/100
45/45 [==============================] - 178s 4s/step - loss: 0.0171 - accuracy: 0.9976 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 

In [ ]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [ ]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1721


In [ ]:
accuracy_10001_labled=(count/len(test_res))*100

In [ ]:
print("Accuracy at 1% labled data and lr=.0001",accuracy_10001_labled)

Accuracy at 1% labled data and lr=.0001 84.033203125


In [ ]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.84      1.00      0.91      1721
        Real       0.00      0.00      0.00       327

    accuracy                           0.84      2048
   macro avg       0.42      0.50      0.46      2048
weighted avg       0.71      0.84      0.77      2048



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Training Model when data is 1% labeled and batch size 256

In [ ]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.1 ,random_state=0)

In [ ]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.99,random_state=0)

In [ ]:
max_features = 10000
maxlen = 200

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [ ]:
batch_size = 256

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate = .001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model=build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 200, 160)          167040    
                                                                 
 lstm_1 (LSTM)               (None, 160)               205440    
                                                                 
 dense (Dense)               (None, 160)               25760     
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 1,398,401
Trainable params: 398,401
Non-trainable params: 1,000,000
_________________________________________________________________


In [ ]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
1/1 [==============================] - 12s 12s/step - loss: 0.6929 - accuracy: 0.5469 - val_loss: 0.6822 - val_accuracy: 0.8571
Epoch 2/300
1/1 [==============================] - 2s 2s/step - loss: 0.6822 - accuracy: 0.8594 - val_loss: 0.6645 - val_accuracy: 0.8571
Epoch 3/300
1/1 [==============================] - 2s 2s/step - loss: 0.6651 - accuracy: 0.8594 - val_loss: 0.6305 - val_accuracy: 0.8571
Epoch 4/300
1/1 [==============================] - 2s 2s/step - loss: 0.6326 - accuracy: 0.8594 - val_loss: 0.5611 - val_accuracy: 0.8571
Epoch 5/300
1/1 [==============================] - 2s 2s/step - loss: 0.5712 - accuracy: 0.8594 - val_loss: 0.4376 - val_accuracy: 0.8571
Epoch 6/300
1/1 [==============================] - 2s 2s/step - loss: 0.4584 - accuracy: 0.8594 - val_loss: 0.4831 - val_accuracy: 0.8571
Epoch 7/300
1/1 [==============================] - 2s 2s/step - loss: 0.4740 - accuracy: 0.8594 - val_loss: 0.4757 - val_accuracy: 0.8571
Epoch 8/300
1/1 [===============

In [ ]:
pred=(model.predict(X_unl) > 0.5).astype("int32")

In [ ]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [ ]:
model=build_model()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
26/26 [==============================] - 147s 5s/step - loss: 0.1712 - accuracy: 0.9705 - val_loss: 0.0446 - val_accuracy: 0.9924
Epoch 2/100
26/26 [==============================] - 139s 5s/step - loss: 0.0587 - accuracy: 0.9898 - val_loss: 0.0446 - val_accuracy: 0.9924
Epoch 3/100
26/26 [==============================] - 139s 5s/step - loss: 0.0587 - accuracy: 0.9898 - val_loss: 0.0450 - val_accuracy: 0.9924
Epoch 4/100
26/26 [==============================] - 139s 5s/step - loss: 0.0597 - accuracy: 0.9898 - val_loss: 0.0464 - val_accuracy: 0.9924
Epoch 5/100
26/26 [==============================] - 139s 5s/step - loss: 0.0580 - accuracy: 0.9898 - val_loss: 0.0456 - val_accuracy: 0.9924
Epoch 6/100
26/26 [==============================] - 139s 5s/step - loss: 0.0580 - accuracy: 0.9898 - val_loss: 0.0492 - val_accuracy: 0.9924
Epoch 7/100
26/26 [==============================] - 138s 5s/step - loss: 0.0590 - accuracy: 0.9898 - val_loss: 0.0463 - val_accuracy: 0.9924
Epoch 

In [ ]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [ ]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

852


In [ ]:
accuracy_1_bs_256_labled=(count/len(test_res))*100

In [ ]:
print("Accuracy at 1% labled data and bs 256",accuracy_1_bs_256_labled)

Accuracy at 1% labled data and bs 256 83.203125


In [ ]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91       852
        Real       0.00      0.00      0.00       172

    accuracy                           0.83      1024
   macro avg       0.42      0.50      0.45      1024
weighted avg       0.69      0.83      0.76      1024



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Training Model when data is 1% labeled and bs=512

In [12]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.1 ,random_state=42)

In [13]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.99,random_state=0)

In [14]:
max_features = 10000
maxlen = 200

In [15]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [16]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [17]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [18]:
batch_size = 512

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate = .001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [20]:
model = build_model()

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 200, 160)          167040    
                                                                 
 lstm_1 (LSTM)               (None, 160)               205440    
                                                                 
 dense (Dense)               (None, 160)               25760     
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 1,398,401
Trainable params: 398,401
Non-trainable params: 1,000,000
_________________________________________________________________


In [22]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
1/1 [==============================] - 16s 16s/step - loss: 0.6940 - accuracy: 0.2969 - val_loss: 0.6832 - val_accuracy: 0.8571
Epoch 2/300
1/1 [==============================] - 2s 2s/step - loss: 0.6843 - accuracy: 0.7969 - val_loss: 0.6670 - val_accuracy: 0.8571
Epoch 3/300
1/1 [==============================] - 2s 2s/step - loss: 0.6697 - accuracy: 0.7969 - val_loss: 0.6361 - val_accuracy: 0.8571
Epoch 4/300
1/1 [==============================] - 2s 2s/step - loss: 0.6409 - accuracy: 0.7969 - val_loss: 0.5721 - val_accuracy: 0.8571
Epoch 5/300
1/1 [==============================] - 2s 2s/step - loss: 0.5862 - accuracy: 0.7969 - val_loss: 0.4667 - val_accuracy: 0.8571
Epoch 6/300
1/1 [==============================] - 2s 2s/step - loss: 0.5095 - accuracy: 0.7969 - val_loss: 0.5085 - val_accuracy: 0.8571
Epoch 7/300
1/1 [==============================] - 2s 2s/step - loss: 0.6185 - accuracy: 0.7969 - val_loss: 0.4578 - val_accuracy: 0.8571
Epoch 8/300
1/1 [===============

In [23]:
pred=(model.predict(X_unl) > 0.50).astype("int32")

In [24]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [25]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
13/13 [==============================] - 144s 11s/step - loss: 0.2418 - accuracy: 0.8995 - val_loss: 0.1480 - val_accuracy: 0.9472
Epoch 2/100
13/13 [==============================] - 132s 10s/step - loss: 0.2022 - accuracy: 0.9186 - val_loss: 0.1540 - val_accuracy: 0.9436
Epoch 3/100
13/13 [==============================] - 131s 10s/step - loss: 0.1876 - accuracy: 0.9236 - val_loss: 0.1470 - val_accuracy: 0.9443
Epoch 4/100
13/13 [==============================] - 131s 10s/step - loss: 0.1860 - accuracy: 0.9234 - val_loss: 0.1446 - val_accuracy: 0.9447
Epoch 5/100
13/13 [==============================] - 131s 10s/step - loss: 0.1809 - accuracy: 0.9278 - val_loss: 0.1433 - val_accuracy: 0.9439
Epoch 6/100
13/13 [==============================] - 131s 10s/step - loss: 0.1760 - accuracy: 0.9305 - val_loss: 0.1392 - val_accuracy: 0.9443
Epoch 7/100
13/13 [==============================] - 131s 10s/step - loss: 0.1805 - accuracy: 0.9254 - val_loss: 0.1350 - val_accuracy: 0.9436

In [26]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [27]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

771


In [28]:
accuracy_1_labled_bs_512=(count/len(test_res))*100

In [29]:
print("Accuracy at 1% labled data and bs=512",accuracy_1_labled_bs_512)

Accuracy at 1% labled data and bs=512 75.29296875


In [30]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.82      0.90      0.86       839
        Real       0.18      0.10      0.13       185

    accuracy                           0.75      1024
   macro avg       0.50      0.50      0.49      1024
weighted avg       0.70      0.75      0.72      1024

